In [1]:
import os
import numpy as np
import datetime
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
from scipy.optimize import minimize
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from curve import LinearCurve, QuadraticCurve, CubicCurve, CubicSplineCurve
from pricer import CurvePricer, SimplePricer, DefaultPricer
from derivative import ZeroCouponBond, CallOnZeroCouponBond, PutOnZeroCouponBond, Caplet, Floorlet, Cap, Floor
from calibration_with_default import calibration, array_to_param, own_calibration, define_optimization_function, squared, generate_bounds
from dcf import dcf_generator

# RNG Seeding
gen_seed, opt_seed = 743453, 894520

path = r'C:/Users/Miikka/PycharmProjects/gradu/'

file = 'germany.csv'
germany = pd.read_csv(path+file, sep=';', header=0)

file = 'italy.csv'
italy = pd.read_csv(path+file, sep=';', header=0)

fun = lambda x: datetime.datetime.strptime(x, '%d.%m.%Y')

germany['Date'] = germany['Date'].apply(fun)
italy['Date'] = italy['Date'].apply(fun)

germany_indexed = germany.set_index(keys='Date')
italy_indexed = italy.set_index(keys='Date')

def modify_germany(germany, germany_indexed):

    length = germany.shape[0]

    ger = {}
    ger_zeros = {}
        
    for i in range(length):
        day = germany.iloc[i, 0].date()
        ger[day] = {}
        
        m6 = day + relativedelta(months=+6)
        y1 = day + relativedelta(years=+1)
        y3 = day + relativedelta(years=+3)
        y5 = day + relativedelta(years=+5)
        y8 = day + relativedelta(years=+8)
        y10 = day + relativedelta(years=+10)
        y20 = day + relativedelta(years=+20)
        y30 = day + relativedelta(years=+30)

        dates = [m6, y1, y3, y5, y8, y10, y20, y30]
        
        rates = pd.Series(germany_indexed.iloc[i, :], name='Rates')
        rates.index = dates
        rates = rates.dropna()
        dates = list(rates.index)
        
        ger[day]['curve'] = CubicSplineCurve(today=day)
        ger[day]['curve'].give_rates(rates/100)

        interpolated = ger[day]['curve'].interpolation(dates=dates)
        zeros = interpolated['Zeros']

        ger[day]['pricer'] = SimplePricer(today=day)
        ger[day]['zeros'] = [ZeroCouponBond(maturity=dates[j],
                                            market_price=price) for j, price in enumerate(zeros)]
        ger_zeros[day]= [zero.calibrate for zero in ger[day]['zeros']]
        
    return ger, ger_zeros

ger, ger_zeros = modify_germany(germany, germany_indexed)


def modify_italy(italy, italy_indexed):

    length = italy.shape[0]

    ita = {}
    ita_zeros = {}

    for i in range(length):
        day = italy.iloc[i, 0].date()
        ita[day] = {}
        
        m6 = day + relativedelta(months=+6)
        y1 = day + relativedelta(years=+1)
        y2 = day + relativedelta(years=+2)
        y3 = day + relativedelta(years=+3)
        y5 = day + relativedelta(years=+5)
        y8 = day + relativedelta(years=+8)
        y10 = day + relativedelta(years=+10)
        y15 = day + relativedelta(years=+15)
        y20 = day + relativedelta(years=+20)
        y30 = day + relativedelta(years=+30)

        dates = [m6, y1, y2, y3, y5, y8, y10, y15, y20, y30]
        
        rates = pd.Series(italy_indexed.iloc[i, :], name='Rates')
        rates.index = dates
        rates = rates.dropna()
        dates = list(rates.index)
        
        ita[day]['curve'] = CubicSplineCurve(today=day)
        ita[day]['curve'].give_rates(rates/100)

        interpolated = ita[day]['curve'].interpolation(dates=dates)
        zeros = interpolated['Zeros']

        ita[day]['pricer'] = SimplePricer(today=day)
        ita[day]['zeros'] = [ZeroCouponBond(maturity=dates[j],
                                            market_price=price) for j, price in enumerate(zeros)]
        ita_zeros[day]= [zero.calibrate for zero in ita[day]['zeros']]
        
    return ita, ita_zeros

ita, ita_zeros = modify_italy(italy, italy_indexed)

bounds = dict(shift=(-0.2, 0.2),
              alpha=(0.0001, 3),
              theta=(0, 0.25),
              sigma=(0.0001, 0.25),
              x0=(0.0001, 0.1),
              k=(0.0001, 3),
              nu=(0.0001, 0.25),
              rho=(-0.99, 0.99),
              y0=(-0.2, 0.2))


In [2]:
first_day = germany['Date'][0].date()

bounds = dict(spread=(-0.2, 0.2),
              alpha=(0.0001, 3),
              theta=(0, 0.25),
              sigma=(0.0001, 0.25),
              x0=(0.0001, 0.1),
              k=(0.0001, 3),
              nu=(0.0001, 0.25),
              rho=(-0.99, 0.99),
              y0=(-0.2, 0.2),
              LGD=(0, 0.6))

def series_calibration(M, N, gen_seed, opt_seed, bounds, LGD, abcd):
    
    res_d = {}
    
    a_m, b_m, c_i, d_i = abcd

    res, res_evo, calibration_object, duration, handler = own_calibration(M=M,
                                                                 N=N,
                                                                 gen_seed=gen_seed,
                                                                 opt_seed=opt_seed,
                                                                 pricer=ger[first_day]['pricer'],
                                                                 riskless_instruments=ger[first_day]['zeros'],
                                                                 risky_instruments=ita[first_day]['zeros'],
                                                                 a_m=a_m,
                                                                 b_m=b_m,
                                                                 c_i=c_i,
                                                                 d_i=d_i,
                                                                 LGD=LGD,
                                                                 bounds=bounds,
                                                                 generational_cycle=50,
                                                                 size=4096,
                                                                 generations=500,
                                                                 return_handler=True)
    
    x0 = res.x

    bnds = handler.generate_bounds(**bounds)

    results = {}
    
    length = germany.shape[0]

    for i in range(length):
        day = germany.iloc[i, 0].date()
        
        pricer = ger[day]['pricer']

        optimization_fun, handler = define_optimization_function(squared,
                                                        M=M, 
                                                        N=N,
                                                        simple=True,
                                                        riskless_instruments=ger[first_day]['zeros'],
                                                        risky_instruments=ita[first_day]['zeros'],
                                                        a_m=a_m,
                                                        b_m=b_m,
                                                        c_i=c_i,
                                                        d_i=d_i,
                                                        LGD=LGD,
                                                        pricer=pricer,
                                                        scale=10**6)

        res = minimize(optimization_fun,
                       x0=x0,
                       bounds=bnds,
                       method='L-BFGS-B')

        res_d[day] = res

        param, def_param = handler.generate_param_dicts(res.x)

        results[day] = param

        x0 = res.x

        print('')
        print(f'{i+1} out of {length} done. {res.fun}')
        print(param)
        print('')
        
    return results, res_d




In [3]:
gen_seed, opt_seed = 84854, 932322

M, N = 1, 2
results12, res_d12 = series_calibration(M, N, gen_seed, opt_seed, bounds, None, ([0],[1],[1],[0]))
"""
M, N = 0, 2
results02, res_d02 = series_calibration(M, N, gen_seed, opt_seed, bounds, None, ([0],[0],[1,0],[0,1]))

M, N = 2, 2
results22, res_d22 = series_calibration(M, N, gen_seed, opt_seed, bounds, None, ([1, 0],[0, 1],[0],[0]))
"""

C:\Users\Miikka\Anaconda3\lib\site-packages\numpy\core\_methods.py:116: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


Generation:     0. Size 4096. Best fitness: 226611.6898. Mean fitness: 11026655478047786684159669232017166635987242856546702837646365671721818129962419151288298602082275035380183344749204035783323296967747274659310970940202849419309518989579916655539778300838426602028192679705795851452416.0000. Diversity:      inf. Update ratio: 0.4130. Strategy: exploration. Seconds:  6.78


C:\Users\Miikka\PycharmProjects\gradu\calibration_with_default.py:665: RuntimeWarning: overflow encountered in double_scalars
  return (x - y) ** 2


Generation:    50. Size 4096. Best fitness: 47369.4049. Mean fitness: 4885832.8221. Diversity:   1.6883. Update ratio: 0.0660. Strategy: exploration. Seconds: 344.65
Generation:   100. Size 1746. Best fitness: 43499.4841. Mean fitness: 460663.0656. Diversity:   0.5121. Update ratio: 0.0170. Strategy: convergence. Seconds: 132.57
Generation:   150. Size  710. Best fitness: 32157.2612. Mean fitness: 249644.9832. Diversity:   0.3571. Update ratio: 0.0240. Strategy: exploration. Seconds: 50.23
Generation:   200. Size  290. Best fitness: 32157.2612. Mean fitness: 151128.7894. Diversity:   0.3883. Update ratio: 0.0070. Strategy: convergence. Seconds: 19.59
Generation:   250. Size  116. Best fitness: 32157.2612. Mean fitness: 80925.8535. Diversity:   0.3422. Update ratio: 0.0090. Strategy: convergence. Seconds:  7.73
Generation:   300. Size   46. Best fitness: 31013.6914. Mean fitness: 50034.6550. Diversity:   0.2307. Update ratio: 0.0220. Strategy: exploration. Seconds:  3.05
Generation:   3


15 out of 180 done. 273.77842697726743
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.07459923184121467, 'spread': 0.0, 'alpha': array([0.41207686]), 'theta': array([0.1359654]), 'sigma': array([0.1420928]), 'X_0': array([0.07957644]), 'k': array([3.]), 'nu': array([0.0001]), 'Y_0': array([0.12334449]), 'a_m': [0], 'b_m': [1], 'c_i': [1], 'd_i': [0], 'LGD': 0.5035287664307279, 'risk_free': True}


16 out of 180 done. 251.31608919347903
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.07683969615980318, 'spread': 0.0, 'alpha': array([0.44994177]), 'theta': array([0.13784582]), 'sigma': array([0.14981377]), 'X_0': array([0.07376118]), 'k': array([1.84769731]), 'nu': array([0.00010217]), 'Y_0': array([0.1011921]), 'a_m': [0], 'b_m': [1], 'c_i': [1], 'd_i': [0], 'LGD': 0.5035287664307279, 'risk_free': True}


17 out of 180 done. 278.3556451139177
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.05033072341336571, 'spread': 0.0, 'alpha': array([0.50144267]), 'theta': array([0.10447011]), 'sigma': array([0.00013

...


102 out of 180 done. 396581.3229053756
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.014300429065173448, 'spread': 0.0, 'alpha': array([0.06891408]), 'theta': array([0.25]), 'sigma': array([0.00081483]), 'X_0': array([0.0001]), 'k': array([0.81429637]), 'nu': array([0.25]), 'Y_0': array([0.2]), 'a_m': [0], 'b_m': [1], 'c_i': [1], 'd_i': [0], 'LGD': 0.5035287664307279, 'risk_free': True}


103 out of 180 done. 396209.58568027057
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.013671803283203906, 'spread': 0.0, 'alpha': array([0.06811012]), 'theta': array([0.25]), 'sigma': array([0.0001]), 'X_0': array([0.0001]), 'k': array([0.80892643]), 'nu': array([0.25]), 'Y_0': array([0.2]), 'a_m': [0], 'b_m': [1], 'c_i': [1], 'd_i': [0], 'LGD': 0.5035287664307279, 'risk_free': True}


104 out of 180 done. 397216.7405376758
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.01354709322956172, 'spread': 0.0, 'alpha': array([0.06822161]), 'theta': array([0.25]), 'sigma': array([0.00040014]), 'X_0': array([0.0001]), 'k': a


124 out of 180 done. 569303.334632088
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.016983088260933404, 'spread': 0.0, 'alpha': array([0.07693224]), 'theta': array([0.25]), 'sigma': array([0.00054247]), 'X_0': array([0.0001]), 'k': array([0.6756519]), 'nu': array([0.25]), 'Y_0': array([0.2]), 'a_m': [0], 'b_m': [1], 'c_i': [1], 'd_i': [0], 'LGD': 0.5035287664307279, 'risk_free': True}


125 out of 180 done. 560431.1487542964
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.015613347086849957, 'spread': 0.0, 'alpha': array([0.07578785]), 'theta': array([0.25]), 'sigma': array([0.00069347]), 'X_0': array([0.0001]), 'k': array([0.66859129]), 'nu': array([0.25]), 'Y_0': array([0.2]), 'a_m': [0], 'b_m': [1], 'c_i': [1], 'd_i': [0], 'LGD': 0.5035287664307279, 'risk_free': True}


126 out of 180 done. 553325.9350844148
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.01449944492868643, 'spread': 0.0, 'alpha': array([0.0750258]), 'theta': array([0.25]), 'sigma': array([0.0002554]), 'X_0': array([0.0001]), 'k': ar


146 out of 180 done. 581120.27377856
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.005109998889121217, 'spread': 0.0, 'alpha': array([0.07454763]), 'theta': array([0.25]), 'sigma': array([0.00028025]), 'X_0': array([0.0001]), 'k': array([0.56604484]), 'nu': array([0.25]), 'Y_0': array([0.2]), 'a_m': [0], 'b_m': [1], 'c_i': [1], 'd_i': [0], 'LGD': 0.5035287664307279, 'risk_free': True}


147 out of 180 done. 587427.5478004783
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.00483576510979939, 'spread': 0.0, 'alpha': array([0.07468435]), 'theta': array([0.25]), 'sigma': array([0.00018916]), 'X_0': array([0.0001]), 'k': array([0.56282726]), 'nu': array([0.25]), 'Y_0': array([0.2]), 'a_m': [0], 'b_m': [1], 'c_i': [1], 'd_i': [0], 'LGD': 0.5035287664307279, 'risk_free': True}


148 out of 180 done. 593813.0725895684
{'M': 1, 'N': 2, 'shift': 0, 'delta': -0.004314909460962221, 'spread': 0.0, 'alpha': array([0.07429856]), 'theta': array([0.25]), 'sigma': array([0.00018977]), 'X_0': array([0.0001]), 'k': 


168 out of 180 done. 783590.5030359554
{'M': 1, 'N': 2, 'shift': 0, 'delta': 0.005877223746808106, 'spread': 0.0, 'alpha': array([0.07010245]), 'theta': array([0.25]), 'sigma': array([0.000152]), 'X_0': array([0.0001]), 'k': array([0.51243058]), 'nu': array([0.25]), 'Y_0': array([0.2]), 'a_m': [0], 'b_m': [1], 'c_i': [1], 'd_i': [0], 'LGD': 0.5035287664307279, 'risk_free': True}


169 out of 180 done. 797161.9091690177
{'M': 1, 'N': 2, 'shift': 0, 'delta': 0.006797374756248123, 'spread': 0.0, 'alpha': array([0.0693738]), 'theta': array([0.25]), 'sigma': array([0.00010167]), 'X_0': array([0.0001]), 'k': array([0.51056265]), 'nu': array([0.25]), 'Y_0': array([0.2]), 'a_m': [0], 'b_m': [1], 'c_i': [1], 'd_i': [0], 'LGD': 0.5035287664307279, 'risk_free': True}


170 out of 180 done. 809918.0183454304
{'M': 1, 'N': 2, 'shift': 0, 'delta': 0.00738801758403548, 'spread': 0.0, 'alpha': array([0.06923631]), 'theta': array([0.25]), 'sigma': array([0.0002966]), 'X_0': array([0.0001]), 'k': array

'\nM, N = 0, 2\nresults02, res_d02 = series_calibration(M, N, gen_seed, opt_seed, bounds, None, ([0],[0],[1,0],[0,1]))\n\nM, N = 2, 2\nresults22, res_d22 = series_calibration(M, N, gen_seed, opt_seed, bounds, None, ([1, 0],[0, 1],[0],[0]))\n'

1Y     2005-03-26
3Y     2007-03-26
5Y     2009-03-26
8Y     2012-03-26
10Y    2014-03-26
20Y    2024-03-26
30Y    2034-03-26
Name: 2004-03-26 00:00:00, dtype: object